In [1]:
import pandas as pd
from datetime import datetime
from test.map_maker import make_map

import json

In [3]:
with open('./map_auction.json') as f:
    jo = json.loads(f.read())

```
 {'acSeCode': 1, 'aucSeNm': '경매',
 'catgoryCode': 12,
 'catgoryNewCode': 12, 'catgoryNewNm': '조미채소류',
 'catgoryNm': '조미채소류', 'cprInsttCode': 35030101,
 'cprInsttNewCode': 6054505, 'cprMtcCode': 570600, 'cprUsePrdlstCode': 5201, 'cprUsePrdlstNm': '대파',
 'delngDe': 20200102, 'delngPrut': 4, 'delngQy': 160, 'insttNewNm': '익산원협(공)', 'insttNm': '익산원협(공)',
 'ledgNo': 56, 'rnum': 196, 'sbidPric': 3800, 'shipmntSeCode': 3,
 'shipmntSeNm': '개별', 'sleSeqn': 10, 'stdFrmlcNewCode': 101,
 'stdFrmlcNewNm': '상자', 'stdMgNewCode': '1ZZ', 'stdMgNewNm': '기타',
 'stdMtcNewCode': 54500, 'stdMtcNewNm': '전라북도 익산시', 'stdPrdlstCode': 1202, 'stdPrdlstNewCode': 1202, 'stdPrdlstNewNm': '대파', 'stdPrdlstNm': '대파', 'stdQlityNewCode': 11, 'stdQlityNewNm': '특', 'stdSpciesCode': 120201, 'stdSpciesNewCode': 120201, 'stdSpciesNewNm': '대파(일반)', 'stdSpciesNm': '대파(일반)', 'stdUnitNewCode': 12, 'stdUnitNewNm': 'kg', 'whsalMrktCode': 350301,
 'whsalMrktNewCode': 1054501, 'whsalMrktNewNm': '익산농수산물도매시장', 'whsalMrktNm': '익산도매시장'}
```

### Design
```
해당 도매시장에서 일부만 있는 경우 -> 없는 데이터는 max시간에 +1씩
해당 날짜 경매 참석한 모든 공판장
모든 공판장별 max, min을 구한다.
```

In [11]:
def get_max_timestamp(p_jo):
    whsal_mrk = {}
    for item in p_jo:
        if whsal_mrk.get(item['whsalMrktNewCode']) == None:
            whsal_mrk[item['whsalMrktNewCode']] = {'max_timestamp':0, 'date':None}

        if item.get('sbidTime') == None:
            # 없는 경우는 당일 08:00:00에서 +1씩 증가
            dt = datetime.strptime(str(item['delngDe']), '%Y%m%d')

        else:
            # 있는 경우는 제대로 없는 경우가 있다
            # ex) '2020-09-22 ::' 이렇게 time을 안찍은 경우가 있음.
            # try catch를 건다.
            # pattern에 match하면 아래 로직 타고 아니면 delngDe로
            try:
                dt = datetime.strptime(str(item['sbidTime']), '%Y-%m-%d %H:%M:%S')
            except Exception as e:
                dt = datetime.strptime(str(item['delngDe']), '%Y%m%d')
            # print(item['whsalMrktNewCode'], item['delngDe'], item['sbidTime'])

        tst = int(datetime.timestamp(dt))
        if tst > whsal_mrk[item['whsalMrktNewCode']]['max_timestamp']:
            whsal_mrk[item['whsalMrktNewCode']]['max_timestamp'] = tst
            whsal_mrk[item['whsalMrktNewCode']]['date'] = datetime.fromtimestamp(tst)

    return whsal_mrk

In [ ]:
def fn1(whsal_mrk):
    # #whsal_mrk를 참고해서 없는 애들은
    for item in jo['20200720']:
        if item.get('sbidTime') == None:
            whsal_mrk[item['whsalMrktNewCode']]['max_timestamp'] += 1
            sbid_tst = whsal_mrk[item['whsalMrktNewCode']]['max_timestamp']
            sbid_time = datetime.fromtimestamp(sbid_tst)
            print(sbid_time)
        else:
            # max에 1초를 더한 값을 바인딩 하고
            # max에 1을 더한 값을 저장한다
            whsal_mrk[item['whsalMrktNewCode']]['max_timestamp'] += 1
            tst = whsal_mrk[item['whsalMrktNewCode']]['max_timestamp']
            sbid_tst = datetime.fromtimestamp(tst)
            print(sbid_tst)

        # print(item['sbidTime'], item['whsalMrktNewCode'], item['whsalMrktNewNm'])

# whsal_mrk = get_max_timestamp(jo['20200922'])
# fn1(whsal_mrk)

In [23]:
# 작물코드 1201는 있고 날짜, wcode만 남음
def save_each_whsal_mrkt():
    for delngDe in jo:
        r = make_map(jo[delngDe], 'whsalMrktCode')
        for whsalMrktCode in r:
            with open(f"./data/{delngDe}_{whsalMrktCode}.json", 'w+') as f:
                f.write(json.dumps(r[whsalMrktCode]))


